# Рекомендательная система. Сем №2

Сайт библиотеки: http://surpriselib.com/

Установки библиотеки:


```$ pip install scikit-surprise ```


или
```
$ conda install -c conda-forge scikit-surprise```

In [1]:
import pandas as pd
import numpy as np

In [2]:
from surprise import Dataset
from surprise import Reader

In [3]:
from surprise import SVD, NMF  

In [4]:
from surprise import KNNBaseline, KNNBasic
from surprise.model_selection import cross_validate

## Задание:
```data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.30, random_state=999)```
1. Для параметров i = [1,5,9,12,17,22] найдите лучший SVD (random_state=999)

2. Сделайте прогноз по всем пользователямKNN (k=15, c корреляцией Пирсона и item-based подходом) и SVD (n = лучший из пункта 1).

### Задача 1

In [5]:
from surprise.model_selection import train_test_split

In [6]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.30, random_state=999)

In [12]:
algo = SVD(n_factors=4,random_state=999, verbose=False) #отключим вывод логирования
predictions = algo.fit(trainset).test(testset)
df_ratings=pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
    df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

In [13]:
df_ratings

,userID,itemID,rating,real_rating
0,753,357,3.743384,4.0
1,542,585,2.609572,2.0
2,5,364,2.364618,1.0
3,887,385,3.974109,4.0
4,383,480,4.700993,5.0
...,...,...,...,...
29995,5,176,3.355142,3.0
29996,155,294,2.692794,3.0
29997,124,1,3.598232,3.0
29998,797,1254,2.030710,2.0


In [20]:
algo.pu

array([[ 0.26933196,  0.10985419, -0.03427613,  0.08596874],
       [-0.03993925,  0.23920804,  0.39710253, -0.08256209],
       [ 0.09742672,  0.10257214, -0.01941669, -0.00504629],
       ...,
       [-0.0866788 ,  0.09452159, -0.00176666,  0.09934645],
       [ 0.15477257,  0.04254256,  0.24489335, -0.20318488],
       [-0.03595732,  0.04401856,  0.19155207, -0.14261572]])

In [14]:
algo.pu[1]

array([-0.03993925,  0.23920804,  0.39710253, -0.08256209])

In [15]:
algo.qi[0]

array([ 0.06452016, -0.02184471, -0.10571095,  0.18161243])

In [16]:
algo.bu

array([-1.75703611e+00, -3.52823602e-01,  3.85997074e-01, -3.93605016e-01,
       -3.92674887e-02, -5.39675717e-01,  8.30955534e-02, -1.55618788e-01,
        1.47563210e-01,  1.78636576e-01,  5.60081348e-01,  1.42491469e-01,
        1.33212436e-01,  5.61539824e-02, -8.79143966e-02, -3.52309080e-01,
       -1.26425792e+00, -1.62783559e-01,  1.56689215e-02,  1.35065534e-01,
        6.10392099e-01, -2.84459264e-01, -1.62188321e-01, -2.29083195e-01,
       -6.02708376e-01, -2.12048864e-01,  2.09549919e-01, -2.96969048e-01,
        2.04411690e-01, -1.61431347e+00, -2.07293849e-01,  1.51269916e-01,
       -2.41588285e-01, -2.62168535e-01, -4.07658843e-01,  1.33444511e-01,
        1.94183642e-01, -7.42117761e-01, -7.53551304e-01, -6.53842163e-01,
        2.35497647e-01,  9.68268957e-02,  4.13256811e-01,  2.68162096e-01,
       -1.69293009e-01, -2.05590958e-01, -4.69210137e-02, -1.12218804e-01,
        3.39773531e-01,  1.43406894e-01,  2.55173968e-01, -2.54382794e-01,
       -8.40181980e-01, -

In [17]:
algo.bi

array([-0.01487578,  0.13655257,  0.07832204, ..., -0.21284639,
       -0.08927716, -0.09255112])

In [18]:
algo.bu[1] + algo.bi[10] + algo.qi[10].T@algo.pu[1]

0.21246123274356768

In [50]:
algo.bu[1] + algo.bi[10] + algo.qi[10].T@algo.pu[1]

-0.25200785054689584

In [19]:
algo.mu

AttributeError: 'SVD' object has no attribute 'mu'

In [22]:
argmin=5
for k in [1,5,9,12,17,22,50]:
    algo = SVD(n_factors=k,random_state=999, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(k)+'-factors SVD:',np.mean(cv['test_rmse']))

1-factors SVD: 0.9436698382914818
5-factors SVD: 0.9438134270878793
9-factors SVD: 0.9424001320538715
12-factors SVD: 0.9430356140971677
17-factors SVD: 0.943822770343209
22-factors SVD: 0.9424909134333905
50-factors SVD: 0.9434132508878026


In [56]:
argmin=5
for k in [20,30, 50, 80, 100]:
    algo = SVD(n_factors=100, n_epochs=k, random_state=999, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(k)+'-n_epochs SVD:',np.mean(cv['test_rmse']))

20-n_epochs SVD: 0.9459065773851281
30-n_epochs SVD: 0.950832718996418
50-n_epochs SVD: 0.972268205142402
80-n_epochs SVD: 0.9826726940763507
100-n_epochs SVD: 0.9873549554282901


In [57]:
for k in [1,4, 10, 14]:
    algo = SVD(n_factors=100, n_epochs=k, random_state=999, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(k)+'-n_epochs SVD:',np.mean(cv['test_rmse']))

1-n_epochs SVD: 1.019271650649503
4-n_epochs SVD: 0.9683529181632808
10-n_epochs SVD: 0.9523492595506995
14-n_epochs SVD: 0.9483583997851031


### Задача 2

In [23]:
from surprise import KNNBaseline

In [24]:
knn=KNNBaseline(k=15,sim_options = {'name': 'pearson_baseline','user_based': False}, verbose=False)
svd = SVD(n_factors=17,random_state=999, verbose=False) 

In [25]:
predictions = knn.fit(trainset).test(testset)
df_ratings=pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
    df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

In [26]:
df_ratings.head()

,userID,itemID,rating,real_rating
0,753,357,3.601156,4.0
1,542,585,2.541421,2.0
2,5,364,2.906434,1.0
3,887,385,4.244959,4.0
4,383,480,4.868237,5.0


In [27]:
predictions = svd.fit(trainset).test(testset)
df_ratings2=pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
    df_ratings2 = df_ratings2.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

In [28]:
for k in [15]:
    knn=KNNBaseline(k=15,sim_options = {'name': 'pearson_baseline','user_based': False}, verbose=False)
    cv=cross_validate(knn, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(k)+'knn:',np.mean(cv['test_rmse']))

15knn: 0.9345118695882918


In [29]:
for k in [15]:
    knn=KNNBasic(k=15,sim_options = {'name': 'pearson_baseline','user_based': False}, verbose=False)
    cv=cross_validate(knn, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(k)+'knn:',np.mean(cv['test_rmse']))

15knn: 1.01332200624872


In [30]:
#Следствие

In [31]:
df_ratings2.head() 

,userID,itemID,rating,real_rating
0,753,357,3.543399,4.0
1,542,585,2.627620,2.0
2,5,364,2.345794,1.0
3,887,385,4.106974,4.0
4,383,480,4.550769,5.0


In [32]:
df_knn_svd=df_ratings.merge(df_ratings2[['userID','itemID','rating']],how='left', on=['userID','itemID'])

In [33]:
df_knn_svd['rating']=(df_knn_svd['rating_x']+ df_knn_svd['rating_y'])/2

In [21]:
df_knn_svd.head()

,userID,itemID,rating_x,real_rating,rating_y,rating
0,753,357,3.601156,4.0,3.543399,3.572277
1,542,585,2.541421,2.0,2.627620,2.584520
2,5,364,2.906434,1.0,2.345794,2.626114
3,887,385,4.244959,4.0,4.106974,4.175967
4,383,480,4.868237,5.0,4.550769,4.709503


## NMF

In [59]:
from surprise import NMF  

In [60]:
argmin=5
for k in [1,4,10,15,20,25,32, 50]:
    algo = NMF(n_factors=k,random_state=999, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(k)+'-factors NMF:',np.mean(cv['test_rmse']))

1-factors NMF: 1.3009509330170606
4-factors NMF: 1.1063728457698445
10-factors NMF: 0.9893727588460131
15-factors NMF: 0.9758209257662086
20-factors NMF: 0.9783083007273139
25-factors NMF: 0.9824531435741221
32-factors NMF: 0.9988008115935831
50-factors NMF: 1.0328863714882648


In [64]:
algo.pu[1]

array([0.19273165, 0.18547481, 1.03501351, 0.12126147, 0.47770598,
       0.09339924, 0.0976314 , 0.0150542 , 0.37456168, 0.74820646,
       0.25646768, 0.15498892, 0.50021891, 0.11870857, 0.01169773,
       0.51504921, 0.1664322 , 0.38964736, 0.47507875, 0.09964147,
       0.59994977, 0.40938974, 0.52482208, 0.04223767, 0.39414989,
       0.45117352, 0.25588527, 0.21995728, 0.91844885, 0.01770147,
       0.49268017, 0.14107989, 0.25107682, 0.01923588, 0.39335371,
       0.19051449, 0.05161039, 0.14090313, 0.62976573, 0.15961137,
       0.19698013, 0.12457982, 0.03719458, 0.16987219, 0.2016178 ,
       0.00842906, 0.6594006 , 0.20935521, 0.07842615, 0.03829879])

In [ ]:
Задача №3
Сравнить NMF, SVD, KNN и их блендинги.
бленд: (SVD + KNN)/2
бленд2: 0.33333... каждому
бленд3: пропорционально RMSE 